### Multiple Jurisdictions ###

With our credible model for dynamic transmission at the contact level and our ability to size environments with approriate contact distributions, we can show the true power of **RKnot** in modelling complex scenarios.

We [have previously shown](load.ipynb#Capacity-Restriction) that contact restrictions and social distancing in any one community *should* be effective in eliminating `sars-cov-2` relatively quickly. Across the world, howvever, many states, provinces, and countries, have experienced second and third waves spaced out months apart despite various forms of strict quarantines and lockdowns.

So why haven't they been effective?

In part, this has resulted from the many exceptions made to these restrictions, in particular, [schools remaining open](load.ipynb#Restriction-for-Schools) in many jurisdictions.

We must also consider the impact of multiple jurisdictions that have unique characteristics, set policy independently, and, importantly, allow travel between them.

We will focus on two jurisdictions to start, building to four, and building in complexity along the way.

To start, we will simply double the size of the environment used in [our dynamic transmission risk models](load.ipynb). Thus,

* n = 20,000
* 8 groups, split evenly into two states: "East State" and "West State"
* the states will be exact reflections of each other: 
    * same size, n = 10,000
    * same demographic mix
    * same number of events
* the event structrue for each state is detailed in [Sizing](sizing.ipynb#Complex-Environments) (and imported from a pickled object as per below).
* each state will have its own vbox. The `East State` vbox must be reassigned and the events adjusted to point to the correct groups.

```python
from copy import deepcopy

from rknot import Sim, Chart
from rknot.events import Restriction, Quarantine
from rknot.dots.fhutch import tmr

group1 = dict(name='W0-19', n=2700, n_inf=0, ifr=0.00003, mover=.98)
group2 = dict(name='W20-49', n=4100, n_inf=1, ifr=0.0002, mover=.98)
group3 = dict(name='W50-69', n=2300, n_inf=1, ifr=0.005, mover=.98)
group4 = dict(name='W70+', n=900, n_inf=0, ifr=0.054, mover=.98)
wstate = [group1, group2, group3, group4]

wbox = {'label': 'W Main', 'box': 344}
wevents = deepcopy(us_w_load_18.events)
wrsxns = deepcopy(us_w_load_18.rsxns)

group5 = dict(name='E0-19', n=2700, n_inf=0, ifr=0.00003, mover=.98)
group6 = dict(name='E20-49', n=4100, n_inf=1, ifr=0.0002, mover=.98)
group7 = dict(name='E50-69', n=2300, n_inf=1, ifr=0.005, mover=.98)
group8 = dict(name='E70+', n=900, n_inf=0, ifr=0.054, mover=.98)
estate =  [group5, group6, group7, group8]

ebox = {'label': 'E Main', 'box': 344}
eevents = deepcopy(us_w_load_18.events)

for e in eevents:
    e.groups = [4,5,6,7]
    e.vbox = 1

groups = wstate + estate
vboxes = [wbox, ebox]
events = wevents + eevents

params = {
    'groups': groups, 'density': 1, 'days': 365, 'tmr_curve': tmr, 
    'vboxes': vboxes, 'events': events
}

sim = Sim(**params)
sim.run()
```

Below, we show the arrangement of all of the dots mixed throughout the space. The only difference from the single jurisdiction structure is the presence of two vboxes (instead of one).

In [25]:
from IPython.core.display import display, HTML
from rknot.notebook import animHTML

display(HTML(animHTML('multi/mixed_agg_dots_only')))

On a proportionate basis, this structure is indistinguishable from the `n=10000`, 4 group structure used in the viral load simulations. And it should result in the same outbreak curve, on average.

In [18]:
from IPython.core.display import display, HTML


import os
from datetime import datetime as dt
import numpy as np

from rknot.helpers import pickload, find_means, find_seconly, find_means_seconly, get_results
from rknot.notebook import curve_table, md
from rknot.helpers import pickload

path = 'vids/multi'
slug = 'mixed-pickled'

file_ = os.path.join(path, slug)
sims = pickload(file_)

last_date = sorted([dt.strptime(date, '%Y-%m-%d') for date in sims.keys()])[-1]
last_date = last_date.strftime('%Y-%m-%d')

res = [get_results(sim) for sim in sims[last_date]['sims']]

headers = ['Mixed', r'Mixed<br>$R_0$>0'] 
m = find_means(res)
m_gt0 = find_means_seconly(res, m)
sizes = [res.shape[0], find_seconly(res).shape[0]]

args = [m, m_gt0]
display(HTML(curve_table(headers, args, sizes)))

As expected, we can see above that, on average, the spread results generated in this structure match those of the [Events structure](load.ipynb#Events) from the viral load analysis.

The curves do tend to have a different shape, however, as per the sample sim below:

In [23]:
from IPython.core.display import display, HTML
from rknot.notebook import animHTML

display(HTML(animHTML('multi/mixed_agg')))

Above, we can see a "wave" effect later in the outbreak. We can seperate the curve among the constituent states to see how each state's curve added to the aggregate.

In [24]:
from IPython.core.display import display, HTML
from rknot.notebook import animHTML

display(HTML(animHTML('multi/mixed_peaks')))

So, from above, we can see that what looked like a single curve and a single outbreak was, in fact, two outbreaks among the two "states", West and East.

What is remarkable is that the dots from both states are free to mix in the main grid. The separate waves result simply from isolated event spaces where only the largest events (3+ capacity) occur within the state groups.

Next we will show the impact of further isolating the two states.


To be continued ...

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [27]:
from rknot.notebook import move_to_doc_folder
move_to_doc_folder('multi.ipynb')